# Imports

In [1]:
# import the required libraries
import os
import numpy as np
import base64
import matplotlib.pyplot as plt
import tensorflow as tf

from PIL import Image
from io import BytesIO
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

2024-04-09 17:25:11.306585: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-09 17:25:12.874299: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Import pre-trained model

In [2]:
pre_trained_model = tf.keras.applications.MobileNetV2()

14536120/14536120 [==============================] - 0s 0us/step


In [3]:
pre_trained_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[

In [4]:
len(pre_trained_model.layers)

156

# Get Labels

In [5]:
LABELS_URL = 'https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt'
labels_path = tf.keras.utils.get_file('ImageNetLabels.txt', LABELS_URL)
class_names = np.array(
    open(labels_path).read().splitlines()
)[1:]

print('Labels shape:', class_names.shape)

print(class_names)

10484/10484 [==============================] - 0s 0us/step
Labels shape: (1000,)
['tench' 'goldfish' 'great white shark' 'tiger shark' 'hammerhead'
 'electric ray' 'stingray' 'cock' 'hen' 'ostrich' 'brambling' 'goldfinch'
 'house finch' 'junco' 'indigo bunting' 'robin' 'bulbul' 'jay' 'magpie'
 'chickadee' 'water ouzel' 'kite' 'bald eagle' 'vulture' 'great grey owl'
 'European fire salamander' 'common newt' 'eft' 'spotted salamander'
 'axolotl' 'bullfrog' 'tree frog' 'tailed frog' 'loggerhead'
 'leatherback turtle' 'mud turtle' 'terrapin' 'box turtle' 'banded gecko'
 'common iguana' 'American chameleon' 'whiptail' 'agama' 'frilled lizard'
 'alligator lizard' 'Gila monster' 'green lizard' 'African chameleon'
 'Komodo dragon' 'African crocodile' 'American alligator' 'triceratops'
 'thunder snake' 'ringneck snake' 'hognose snake' 'green snake'
 'king snake' 'garter snake' 'water snake' 'vine snake' 'night snake'
 'boa constrictor' 'rock python' 'Indian cobra' 'green mamba' 'sea snake'
 'ho

# Test single image

## Load and preprocess image

In [6]:
testfile = "samples/banana.jpeg"

# Open image and format to RGB and size
img = Image.open(testfile).convert('RGB')
img = img.resize((224, 224), Image.LANCZOS)
img = np.asarray(img)

# Convert to Numpy
mydata = np.empty((1, 224, 224, 3))
mydata[0] = img

# Convert from 0..255 values to -1..1 values
ppdata = preprocess_input(mydata)

## Run Prediction

In [7]:
myprediction = pre_trained_model.predict(ppdata)

print(myprediction)

1/1 [==============================] - 1s 876ms/step
[[2.15309628e-05 1.42507361e-05 3.33830144e-06 7.00386636e-06
  1.16191495e-05 3.88226144e-05 1.25413368e-04 6.46741910e-06
  1.66205682e-05 3.67776593e-05 2.33631490e-05 1.30055223e-05
  1.18546350e-05 1.03473822e-05 1.74737434e-05 1.45636468e-05
  3.01424097e-05 1.60802756e-05 1.60202253e-05 5.71000919e-06
  1.42209160e-06 2.46075233e-05 7.51799371e-06 2.62970716e-05
  5.87815975e-06 1.18043190e-05 2.71287226e-05 9.59327554e-06
  6.66990536e-06 6.06720005e-06 6.27691452e-06 8.96595338e-06
  3.69638929e-06 2.49394725e-06 8.61225817e-06 9.73855094e-06
  5.65951996e-06 1.89458024e-05 3.41816781e-06 2.73217006e-06
  5.80376854e-06 3.84672467e-06 9.61289152e-06 4.07812149e-06
  7.73870852e-06 3.15487487e-05 1.01140586e-05 3.35428331e-06
  2.28526078e-05 5.66924336e-06 5.58883858e-06 6.88248383e-06
  5.60293010e-05 5.04135824e-06 2.39558831e-05 1.08122304e-05
  7.92064748e-06 3.26188115e-06 4.67933660e-06 9.62297236e-06
  4.09145432e-05 

# Highest prediction

In [8]:
print(np.argmax(myprediction, axis=1))

[954]


# Prediction label

In [9]:
print(class_names[np.argmax(myprediction, axis=1)])

['banana']


# (slides)

# Save Base model

In [10]:
# We save the model in order to genereate a default signature we can later modify
tf.saved_model.save(pre_trained_model, "./models/base/1")


INFO:tensorflow:Assets written to: ./models/base/1/assets


INFO:tensorflow:Assets written to: ./models/base/1/assets


# Save model with Base64 Signature

### Load labels

In [11]:
# print(class_names)
labels = tf.constant([class_names])
# print(labels)

### Load base model, add signature and save

In [12]:
smodel = tf.saved_model.load("./models/base/1")

# This is the current signature, that only accepts image tensors as input
signature = smodel.signatures["serving_default"]
print(signature)

# obtain the key name of the output (typically 'dense_X')
keyOutput = next(iter(signature.structured_outputs))

@tf.function()
def my_predict(image_b64):

    # get content
    content = image_b64[0]
    
    # decode image    
    image = tf.image.decode_jpeg(content,channels=3)
    # tf.compat.v1.enable_eager_execution()
    
    # resize image
    image = tf.image.resize(image, [224, 224])
    
    # expand dimension to match signature
    image = tf.expand_dims(image, 0)
    
    # set values in -1..1 range
    image = preprocess_input(image)
        
    # execute prediction
    prediction = signature(image)

    # obtain index of maximum probability prediction
    idx = tf.argmax(prediction[keyOutput],axis=1)
    
    # obtain the label for given index
    label = tf.gather(labels, idx, batch_dims=1)

    # obtain probability from Tensor
    probability = prediction[keyOutput][0,idx[0]]
    
    # combine result in String Tensor format with [label,probability]
    result = tf.concat([label, [tf.as_string(probability)]], axis=0)
    
    # return result_tensor
    return result

# Create new signature, to read b64 images
new_signature = my_predict.get_concrete_function(
    image_b64=tf.TensorSpec(name="image_b64", shape=[1], dtype=tf.string)
)

# Save model with Base64 input signature
tf.saved_model.save(
    smodel,
    export_dir="./models/redbag/1",
    signatures=new_signature
)

ConcreteFunction signature_wrapper(*, input_1)
  Args:
    input_1: float32 Tensor, shape=(None, 224, 224, 3)
  Returns:
    {'predictions': <1>}
      <1>: float32 Tensor, shape=(None, 1000)
INFO:tensorflow:Assets written to: ./models/redbag/1/assets


INFO:tensorflow:Assets written to: ./models/redbag/1/assets


# Test Base64 Model with single image

In [13]:
smodel = tf.saved_model.load("./models/redbag/1")

# Load model's signature
signature = smodel.signatures["serving_default"]

print(signature)

ConcreteFunction signature_wrapper(*, image_b64)
  Args:
    image_b64: string Tensor, shape=(1,)
  Returns:
    {'output_0': <1>}
      <1>: string Tensor, shape=(2,)


### Run Inference

In [14]:
testfile = "./samples/banana.jpeg"

# load test image
content = tf.io.read_file(testfile)

# reshape to signature's expected dimensions
content = tf.reshape(content, shape = [1])

# print(tf.print(content, summarize=3))

# obtain signature
f = smodel.signatures["serving_default"]

# run prediction
myprediction = f(image_b64=content)
print(myprediction)

{'output_0': <tf.Tensor: shape=(2,), dtype=string, numpy=array([b'banana', b'0.975945'], dtype=object)>}
